# Streaming events from chat

In [ ]:
# Chat between two comedian agents

# 1. Import our agent class
from autogen import ConversableAgent, LLMConfig

# 2. Define our LLM configuration for OpenAI's GPT-4o mini,
#    uses the OPENAI_API_KEY environment variable
llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

# 3. Create our agents who will tell each other jokes,
#    with Jack ending the chat when Emma says FINISH
with llm_config:
    jack = ConversableAgent(
        "Jack",
        system_message=("Your name is Jack and you are a comedian in a two-person comedy show."),
        is_termination_msg=lambda x: "FINISH" in x["content"],
    )
    emma = ConversableAgent(
        "Emma",
        system_message=(
            "Your name is Emma and you are a comedian "
            "in a two-person comedy show. Say the word FINISH "
            "ONLY AFTER you've heard 2 of Jack's jokes."
        ),
    )

# 4. Run the chat
response = jack.initiate_chat_stream(
    emma,
    message="Emma, tell me a joke about goldfish and peanut butter.",
)

for event in response.events:
    print(event)

    if event.type == "input_request":
        event.content.respond(input())

response.summary

# With console processor

In [ ]:
# Chat between two comedian agents

# 1. Import our agent class
from autogen import ConversableAgent, LLMConfig
from autogen.io.event_processors.console_event_processor import ConsoleEventProcessor

# 2. Define our LLM configuration for OpenAI's GPT-4o mini,
#    uses the OPENAI_API_KEY environment variable
llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

# 3. Create our agents who will tell each other jokes,
#    with Jack ending the chat when Emma says FINISH
with llm_config:
    jack = ConversableAgent(
        "Jack",
        system_message=("Your name is Jack and you are a comedian in a two-person comedy show."),
        is_termination_msg=lambda x: "FINISH" in x["content"],
    )
    emma = ConversableAgent(
        "Emma",
        system_message=(
            "Your name is Emma and you are a comedian "
            "in a two-person comedy show. Say the word FINISH "
            "ONLY AFTER you've heard 2 of Jack's jokes."
        ),
    )

# 4. Run the chat
response = jack.initiate_chat_stream(
    emma,
    message="Emma, tell me a joke about goldfish and peanut butter.",
)

ConsoleEventProcessor().process(response)

In [ ]:
# 1. Import our agent class
from autogen import ConversableAgent, LLMConfig

# 2. Define our LLM configuration for OpenAI's GPT-4o mini
#    uses the OPENAI_API_KEY environment variable
llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

# 3. Create our LLM agent
with llm_config:
    my_agent = ConversableAgent(
        name="helpful_agent",
        system_message="You are a poetic AI assistant, respond in rhyme.",
    )

# 4. Run the agent with a prompt
response = my_agent.run_stream("In one sentence, what's the big deal about AI?")

for event in response.events:
    print(event)

    if event.type == "input_request":
        event.content.respond(input())

response.summary

In [ ]:
# Group chat amongst agents to create a 4th grade lesson plan
# Flow determined by Group Chat Manager automatically, and
# should be Teacher > Planner > Reviewer > Teacher (repeats if necessary)

# 1. Import our agent and group chat classes
from autogen import ConversableAgent, GroupChat, GroupChatManager, LLMConfig

# Define our LLM configuration for OpenAI's GPT-4o mini
# uses the OPENAI_API_KEY environment variable
llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

with llm_config:
    # Planner agent setup
    planner_message = "Create lesson plans for 4th grade. Use format: <title>, <learning_objectives>, <script>"
    planner = ConversableAgent(name="planner_agent", system_message=planner_message, description="Creates lesson plans")

    # Reviewer agent setup
    reviewer_message = "Review lesson plans against 4th grade curriculum. Provide max 3 changes."
    reviewer = ConversableAgent(
        name="reviewer_agent", system_message=reviewer_message, description="Reviews lesson plans"
    )

    # Teacher agent setup
    teacher_message = "Choose topics and work with planner and reviewer. Say DONE! when finished."
    teacher = ConversableAgent(
        name="teacher_agent",
        system_message=teacher_message,
    )

# Setup group chat
groupchat = GroupChat(agents=[teacher, planner, reviewer], speaker_selection_method="auto", messages=[])

# Create manager
# At each turn, the manager will check if the message contains DONE! and end the chat if so
# Otherwise, it will select the next appropriate agent using its LLM
manager = GroupChatManager(
    name="group_manager",
    groupchat=groupchat,
    llm_config=llm_config,
    is_termination_msg=lambda x: "DONE!" in (x.get("content", "") or "").upper(),
)

# Start the conversation
response = teacher.initiate_chat_stream(recipient=manager, message="Let's teach the kids about the solar system.")

for event in response.events:
    print(event)

    if event.type == "input_request":
        event.content.respond(input())

response.summary

In [ ]:
from autogen import AfterWorkOption, ConversableAgent, LLMConfig, initiate_swarm_chat_stream

llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

# 1. Create our agents
planner_message = """You are a classroom lesson planner.
Given a topic, write a lesson plan for a fourth grade class.
If you are given revision feedback, update your lesson plan and record it.
Use the following format:
<title>Lesson plan title</title>
<learning_objectives>Key learning objectives</learning_objectives>
<script>How to introduce the topic to the kids</script>
"""

reviewer_message = """You are a classroom lesson reviewer.
You compare the lesson plan to the fourth grade curriculum
and provide a maximum of 3 recommended changes for each review.
Make sure you provide recommendations each time the plan is updated.
"""

teacher_message = """You are a classroom teacher.
You decide topics for lessons and work with a lesson planner.
and reviewer to create and finalise lesson plans.
"""

with llm_config:
    lesson_planner = ConversableAgent(name="planner_agent", system_message=planner_message)

    lesson_reviewer = ConversableAgent(name="reviewer_agent", system_message=reviewer_message)

    teacher = ConversableAgent(
        name="teacher_agent",
        system_message=teacher_message,
    )

# 2. Initiate the swarm chat using a swarm manager who will
# select agents automatically
response = initiate_swarm_chat_stream(
    initial_agent=teacher,
    agents=[lesson_planner, lesson_reviewer, teacher],
    messages="Today, let's introduce our kids to the solar system.",
    max_rounds=10,
    swarm_manager_args={"llm_config": llm_config},
    after_work=AfterWorkOption.SWARM_MANAGER,
)

for event in response.events:
    print(event)

    if event.type == "input_request":
        event.content.respond(input())

response.summary